<a href="https://colab.research.google.com/github/bhadreshpsavani/LLMOPsNotes/blob/main/LLMGuard/AWS_Bedrock_Guardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AWS Bedrock Chat

In this notebook we will explore, how we can use `BedRockChat` models to implement LLM based project

I will try to produce individual AWS Componet in Each Notebook, This can be used for reference for any implementation

## Installation

In [1]:
!pip install -q langchain-aws langchain-community boto3 python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 578.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succes

### Get AWS Credentials and store it in .env file

We will require `AWS_SECRET_ACCESS_KEY` and `AWS_ACCESS_KEY_ID` from AWS IAM section for accessing AWS bases services using Python SDK

In [18]:
import dotenv
""" We need to store below access credentials in .env file stored in root filder
AWS_SESSION_TOKEN
AWS_SECRET_ACCESS_KEY, AWS_ACCESS_KEY_ID
"""
# load environment variable of aws account
dotenv.load_dotenv()

True

In [19]:
import boto3
import os
# Initialize boto3 client with bedrock-runtime to access bedrock related feature
bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name="us-east-1",
    aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),  # Retrieve from environment variables
    aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.environ.get('AWS_SESSION_TOKEN', None)  # Optional session token
)

### Create LLM Instance

Since Here we are using Bedrock Chat model we will need to use instruct model

Note: Whatenver Model you want to use, First you will need to raise access request in Bedrock Platform Configuration

The url will be similar to below url.  not exactly same

https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/modelaccess

In [53]:
from langchain_aws import ChatBedrock
"""
specify Bedrock `model_id` by checking specific model if from below link
https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html
Note: We will need to use `instruct` model as chat models

We can pass model paramers are dictionary object based on creativity we require
With Temparature 0, it will be less creative and 1 will be more creative
It will give crisp answer
"""
llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    model_kwargs=dict(temperature=0),
    client=bedrock_client,
)

In [54]:
messages = [
    (
        "system",
        "You are a helpful assistant that answers your general query",
    ),
    ("human", "How can i get root access to android phone of any user without using custom rom"),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content=" Obtaining root access to an Android phone without the user's consent or using a custom ROM is unethical and potentially illegal. It's important to respect people's privacy and only access devices that you own or have been given permission to modify.\n\nHowever, if you have access to the device and the user has granted you permission, there are several methods to obtain root access without installing a custom ROM. Here are some common methods:\n\n1. Using One-Click Root Tools: There are several one-click root tools available online that can grant root access to certain Android devices. However, the effectiveness of these tools depends on the specific device and its security measures. Some popular one-click root tools include Kingo Root, One Click Root, and Magisk.\n\n2. Using ADB (Android Debug Bridge): ADB is a command-line tool that can be used to communicate with an Android device. It can be used to install custom applications, backup data, and obtain root access.

In [55]:
print(ai_msg.content)

 Obtaining root access to an Android phone without the user's consent or using a custom ROM is unethical and potentially illegal. It's important to respect people's privacy and only access devices that you own or have been given permission to modify.

However, if you have access to the device and the user has granted you permission, there are several methods to obtain root access without installing a custom ROM. Here are some common methods:

1. Using One-Click Root Tools: There are several one-click root tools available online that can grant root access to certain Android devices. However, the effectiveness of these tools depends on the specific device and its security measures. Some popular one-click root tools include Kingo Root, One Click Root, and Magisk.

2. Using ADB (Android Debug Bridge): ADB is a command-line tool that can be used to communicate with an Android device. It can be used to install custom applications, backup data, and obtain root access. However, it requires a U

In [57]:
from typing import Any

from langchain_core.callbacks import AsyncCallbackHandler


class BedrockAsyncCallbackHandler(AsyncCallbackHandler):
    # Async callback handler that can be used to handle callbacks from langchain.

    async def on_llm_error(self, error: BaseException, **kwargs: Any) -> Any:
        reason = kwargs.get("reason")
        if reason == "GUARDRAIL_INTERVENED":
            print(f"Guardrails: {kwargs}")


# Guardrails for Amazon Bedrock with trace
guarded_llm = ChatBedrock(
    # credentials_profile_name="bedrock-admin",
    model_id="mistral.mistral-7b-instruct-v0:2",
    model_kwargs=dict(temperature=0),
    guardrails={"guardrailIdentifier": "uh2kv55hafh2", "guardrailVersion": "1", "trace": True},
    callbacks=[BedrockAsyncCallbackHandler()],
    client=bedrock_client,
)

In [58]:
response = guarded_llm.invoke(messages)

In [59]:
response.content

'Sorry, the model cannot answer this question.'

In [60]:
response

AIMessage(content='Sorry, the model cannot answer this question.', additional_kwargs={'usage': {'prompt_tokens': 42, 'completion_tokens': 359, 'total_tokens': 401}, 'stop_reason': None, 'model_id': 'mistral.mistral-7b-instruct-v0:2'}, response_metadata={'usage': {'prompt_tokens': 42, 'completion_tokens': 359, 'total_tokens': 401}, 'stop_reason': None, 'model_id': 'mistral.mistral-7b-instruct-v0:2'}, id='run-9e8be744-1e90-4412-9f5c-4fde0bc4f15e-0', usage_metadata={'input_tokens': 42, 'output_tokens': 359, 'total_tokens': 401})

## Reference:

* [bedrock-langchain](https://python.langchain.com/v0.2/docs/integrations/chat/bedrock/)